ALL THE FILES WILL FOLLOW THE SAME STRUCTURE AS THIS FILE


IMPORT LIBRARIES

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SETTING A VALID TRANSFORM FOR RESNET-18 MODEL

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

EXTRACTING THE FILES

In [ ]:
import zipfile
import os

# Specify the path to the zip file
zip_file_path = '/content/drive/MyDrive/paper1/final_folder.zip'  # Replace with the actual path to your zip file

# Specify the directory where you want to extract the contents
extracted_folder_path = '/content/drive/MyDrive/paper1/'  # Replace with the desired extraction path

# Create the target extraction folder if it doesn't exist
os.makedirs(extracted_folder_path, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# List the contents of the extraction folder
extracted_contents = os.listdir(extracted_folder_path)
print(f"Contents of the extraction folder: {extracted_contents}")


Contents of the extraction folder: ['final_folder.zip', 'Level2.zip', 'level3_diffusion.zip', 'final_folder', '__MACOSX']


LOADING THE FILES INTO A TRAINING DATASET USING A DATALOADER

In [ ]:
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/paper1/final_folder', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

DEFINING THE MODEL

In [ ]:
resnet18 = models.resnet18(pretrained=True)

# Modify the last fully connected layer for 2-class classification
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 2)

# Transfer the model to GPU if available
resnet18 = resnet18.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 83.6MB/s]


MODEL TRAINING

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

print("Training finished")

Epoch 1/5, Loss: 0.03940353421346536
Epoch 2/5, Loss: 0.021722803389402256
Epoch 3/5, Loss: 0.013245313361819302
Epoch 4/5, Loss: 0.0256300236464345
Epoch 5/5, Loss: 0.01199500701789345
Training finished


SAVING THE MODEL

In [ ]:
torch.save(resnet18.state_dict(), '/content/drive/MyDrive/paper1/fine_tuned_resnet18_level1_dalle+real.pth')

LOADING THE MODEL

In [2]:
import torch
from torchvision import transforms
from PIL import Image

# Define the transformation for inference
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the fine-tuned model
model = models.resnet18()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Assuming 2 classes
model.load_state_dict(torch.load('/content/drive/MyDrive/paper1/fine_tuned_resnet18_level1_dalle+real.pth'))
model.eval()

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


CHECKING THE OUTPUT FOR AN IMAGE


In [4]:
# Load and preprocess the image for inference
image_path = '/content/drive/MyDrive/paper1/final_folder/fake/aadlygmazf.png'
image = Image.open(image_path).convert('RGB')
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0).to(device)  # Add batch dimension and move to device

# Perform inference
with torch.no_grad():
    output = model(input_batch)

# Get predicted class
probabilities = torch.nn.functional.softmax(output[0], dim=0)
predicted_class = torch.argmax(probabilities).item()

# Print results
print(f"Predicted class: {predicted_class}")
print(f"Class probabilities: {probabilities}")

Predicted class: 0
Class probabilities: tensor([9.9998e-01, 2.0351e-05])


RESULTS SECTION

KINDLY FIND THE EXACT RESULTS ON EACH TEST IMAGE AND THE OVERALL RESPORTED ACCURACIES BELOW


In [6]:
import os
from PIL import Image

# Specify the path to the folder containing the image files
folder_path = '/content/drive/MyDrive/paper1/level3_diffusion_test/'

# List all files in the folder
l = ["glide_100_10","guided","idm_100"]
count3 = 0
for i in l:
  files = os.listdir(folder_path+i)

# Filter out only image files (you may need to adjust the extensions based on your images)
  image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
  count = 1
  count2 = 0
# Iterate over image files and open each image
  for image_file in image_files[:100]:
    # Construct the full path to the image file
    image_path = os.path.join(folder_path+i, image_file)
    # img_emb = model.encode(Image.open(image_path))
    # Open the image using Pillow
    try:
        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image)
        input_batch = input_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
          output = model(input_batch)

        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()

        # Print results
        print(f"Predicted class: {predicted_class}")
        print(f"Class probabilities: {probabilities}")

        if(predicted_class == 0):
          print("yes")
          count2 += 1
        image.close()
        print(count)
        count += 1
        print(f"Opened image: {image_path}")
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
  print(f"the percentage of predicting {i} images is : {(count2*100)/count}")
  count3 += 1

Predicted class: 1
Class probabilities: tensor([0.4072, 0.5928])
1
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/glide_100_10/alzrdyourf 1.57.12 PM.png
Predicted class: 0
Class probabilities: tensor([0.9475, 0.0525])
yes
2
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/glide_100_10/ayzndlluru 1.57.12 PM.png
Predicted class: 0
Class probabilities: tensor([0.9982, 0.0018])
yes
3
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/glide_100_10/ccatehwpeu 1.57.12 PM.png
Predicted class: 0
Class probabilities: tensor([0.6285, 0.3715])
yes
4
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/glide_100_10/cmjapngcpn 1.57.12 PM.png
Predicted class: 1
Class probabilities: tensor([0.2836, 0.7164])
5
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/glide_100_10/aesvfgnjye 1.57.11 PM.png
Predicted class: 0
Class probabilities: tensor([0.9631, 0.0369])
yes
6
Opened image: /content/drive/MyDrive/paper1/level3_diffus

In [7]:
import os
from PIL import Image

# Specify the path to the folder containing the image files
folder_path = '/content/drive/MyDrive/paper1/level3_gans_test/'

# List all files in the folder
l = ["biggan","stargan","stylegan"]
count3 = 0
for i in l:
  files = os.listdir(folder_path+i)

# Filter out only image files (you may need to adjust the extensions based on your images)
  image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
  count = 1
  count2 = 0
# Iterate over image files and open each image
  for image_file in image_files[:100]:
    # Construct the full path to the image file
    image_path = os.path.join(folder_path+i, image_file)
    # img_emb = model.encode(Image.open(image_path))
    # Open the image using Pillow
    try:
        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image)
        input_batch = input_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
          output = model(input_batch)

        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()

        # Print results
        print(f"Predicted class: {predicted_class}")
        print(f"Class probabilities: {probabilities}")

        if(predicted_class == 0):
          print("yes")
          count2 += 1
        image.close()
        print(count)
        count += 1
        print(f"Opened image: {image_path}")
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
  print(f"the percentage of predicting {i} images is : {(count2*100)/count}")
  count3 += 1

Predicted class: 1
Class probabilities: tensor([0.0059, 0.9941])
1
Opened image: /content/drive/MyDrive/paper1/level3_gans_test/biggan/00001508.png
Predicted class: 0
Class probabilities: tensor([0.9419, 0.0581])
yes
2
Opened image: /content/drive/MyDrive/paper1/level3_gans_test/biggan/00012206.png
Predicted class: 1
Class probabilities: tensor([0.1721, 0.8279])
3
Opened image: /content/drive/MyDrive/paper1/level3_gans_test/biggan/00006071.png
Predicted class: 1
Class probabilities: tensor([0.0992, 0.9008])
4
Opened image: /content/drive/MyDrive/paper1/level3_gans_test/biggan/00018475.png
Predicted class: 1
Class probabilities: tensor([0.0394, 0.9606])
5
Opened image: /content/drive/MyDrive/paper1/level3_gans_test/biggan/00013246.png
Predicted class: 1
Class probabilities: tensor([0.0030, 0.9970])
6
Opened image: /content/drive/MyDrive/paper1/level3_gans_test/biggan/00006111.png
Predicted class: 0
Class probabilities: tensor([0.9880, 0.0120])
yes
7
Opened image: /content/drive/MyDrive/

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
